In [ ]:
# !pip installpip transformers

In [ ]:
# !pip install pytorch_lightning

In [ ]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset

In [2]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")
# Chatbot_Data = Chatbot_Data.drop(labels='label', axis=1)

# Test 용으로 300개 데이터만 처리한다.
# Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

In [6]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
PAD = "<pad>"
MASK = "<unused0>"
# SENT = '<unused1>'

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2", 
    bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)

"""
koGPT2_TOKENIZER는 transformers의 Character BPE tokenizer(생각하기에는 Byte-level BPE 이거 같다.)로 학습되었습니다.


Huggingface transformers 라이브러리에서는 크게 두 가지 종류의 tokenizer를 지원하는데, 
첫 번째로는 파이썬으로 구현된 일반 tokenizer와 Rust 로 구축된 "Fast" tokenizer로 구분할 수 있다.
"Fast" tokenizer에서는 batched tokenization에서 속도를 더 빠르게 해주고, 입력으로 주어진 문장과 token 사이를 mapping 해주는 추가적인 함수를 지원한다. 
참고로, "Fast" tokenizer들을 SentencePiece tokenizer(T5, Albert, XLNet 등) 들을 지원하지 않고 있다.

Tokenizer들은 크게 3가지 기능을 제공한다.

1. Tokenizing : 입력 문자열을 token id로 변환(encoding), token id를 다시 문자열로 변환(decoding)의 기능

2. 기존의 구조(BPE, Sentencepiece 등)에 독립적으로 추가적인 token들을 추가하는 기능

3. Special token(mask, BOS, EOS 등) 을 관리하는 기능

"""

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [7]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        # self.first = TRUE   ## 연속성 대화 데이터셋 사용 경우
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        # self.sent_token = SENT
        self.bos = BOS
        self.eos = EOS
        self.mask = MASK
        self.pad = PAD
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        # 감정 칼럼을 사용할 경우
        # sent = str(turn['label'])

        q_toked = self.tokenizer.tokenize(self.q_token + q)
        # q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token + sent)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # 데이터가 연속성일 경우 사용
        # if self.first:
        #     logging.info("contexts : {}".format(q))
        #     logging.info("toked ctx: {}".format(q_toked))
        #     logging.info("response : {}".format(a))
        #     logging.info("toked response : {}".format(a_toked))
        #     logging.info('labels {}'.format(labels))
        #     self.first = False

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [8]:
# class KoGPT2Chat(LightningModule):
#     def __init__(self, hparams, **kwargs):
#         super(KoGPT2Chat, self).__init__()
#         self.hparams = hparams
#         self.neg = -1e18
#         self.kogpt2 = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
#         self.loss_function = torch.nn.CrossEntropyLoss(reduction='none')

#     def configure_optimizers(self):
#         # Prepare optimizer
#         param_optimizer = list(self.named_parameters())
#         no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
#         optimizer_grouped_parameters = [
#             {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#             {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#         ]
#         optimizer = AdamW(optimizer_grouped_parameters,
#                           lr=self.hparams.lr, correct_bias=False)
#         # warm up lr
#         num_train_steps = len(self.train_dataloader()) * self.hparams.max_epochs
#         num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
#         scheduler = get_cosine_schedule_with_warmup(
#             optimizer,
#             num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
#         lr_scheduler = {'scheduler': scheduler, 'name': 'cosine_schedule_with_warmup',
#                         'monitor': 'loss', 'interval': 'step',
#                         'frequency': 1}
#         return [optimizer], [lr_scheduler]

In [9]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.cuda.LongTensor(data), torch.cuda.LongTensor(mask), torch.cuda.LongTensor(label)

In [10]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(
    train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch)
"""
*dataset ( Dataset ) :
데이터를 로드할 데이터세트입니다.

*batch_size ( int , optional ) : 
로드할 배치당 샘플 수(기본값: 1).

*shuffle ( bool , optional ) : 
True모든 epoch에서 데이터를 다시 섞도록 설정합니다(기본값: False).

sampler ( Sampler 또는 Iterable , 선택 사항 ) :
데이터 세트에서 샘플을 추출하는 전략을 정의합니다. 구현 Iterable된 모든 것이 가능합니다 . __len__지정된 경우 지정 shuffle하지 않아야 합니다.

batch_sampler ( Sampler 또는 Iterable , 선택 사항 ) :
와 비슷 sampler하지만 한 번에 인덱스 배치를 반환합니다. batch_size, shuffle, sampler및 과( 와) 상호 배타적입니다 drop_last.

*num_workers ( int , optional ) :
데이터 로드에 사용할 하위 프로세스 수. 0데이터가 메인 프로세스에서 로드됨을 의미합니다. (기본값: 0)

*collate_fn ( callable , optional ) :
샘플 목록을 병합하여 Tensor의 미니 배치를 형성합니다. 지도 스타일 데이터세트에서 일괄 로드를 사용할 때 사용됩니다.

pin_memory ( bool , optional ) :
True인 경우 데이터 로더는 Tensor를 반환하기 전에 장치/CUDA 고정 메모리에 복사합니다. 
데이터 요소가 사용자 지정 유형이거나 collate_fn반환이 사용자 지정 유형인 경우 아래 예를 참조하십시오.

drop_last ( bool , optional ) :
True데이터 세트 크기가 배치 크기로 나눌 수 없는 경우 마지막 불완전한 배치를 삭제하도록 설정합니다.
False데이터 세트의 크기가 배치 크기로 나눌 수 없는 경우 마지막 배치는 더 작아집니다 . (기본값: False)

timeout ( 숫자 , 선택 사항 ) :
양수인 경우 작업자로부터 배치를 수집하기 위한 시간 초과 값입니다. 항상 음수가 아니어야 합니다. (기본값: 0)

worker_init_fn ( callable , optional ) :
그렇지 않은 경우 시드 후 및 데이터 로드 전에 작업자 ID( int in )를 입력으로 None사용하여 각 작업자 하위 프로세스에서 호출됩니다 . (기본값: )[0, num_workers - 1]None

generator ( torch.Generator , 선택 사항 ) :
그렇지 않은 경우 이 RNG는 RandomSampler에서 무작위 인덱스를 생성하고 다중 처리를 통해 작업자에 대한 base_seedNone 를 생성하는 데 사용됩니다 . (기본값: )None

prefetch_factor ( int , optional , keyword-only arg ) :
각 작업자가 미리 로드한 배치 수입니다. 2모든 작업자에 대해 총 2 * num_workers 배치가 미리 페치됨을 의미합니다. (기본값: 2)

Persistent_workers ( bool , optional ) :
True인 경우 데이터 로더는 데이터 세트가 한 번 소비된 후 작업자 프로세스를 종료하지 않습니다. 
이를 통해 작업자 데이터 세트 인스턴스를 활성 상태로 유지할 수 있습니다. (기본값: False)

pin_memory_device ( str , optional ) :
데이터 로더는 pin_memory가 true로 설정된 경우 반환하기 전에 Tensor를 장치 고정 메모리에 복사합니다.
"""

In [ ]:
model.to(device)
model.train()

In [13]:
## 조정할 파라미터 구간

learning_rate = 3e-5
Sneg = -1e18

criterion = torch.nn.CrossEntropyLoss(reduction="none")

# configure_optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.NAdam(model.parameters(), lr=learning_rate)

epoch = 10

In [14]:
# training_step, forward

print ("start")
for epoch in range(epoch):
    for batch_idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = batch
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

start


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """


end


In [15]:
model.to('cpu')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + A_TKN + a)).unsqueeze(dim=0) 
            # input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0) 
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(
                    torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))


Chatbot > 저도요
Chatbot > 좋은 아침이에요
Chatbot > 그건 습관이에요
Chatbot > 그 사람도 그럴 거예요
